In [2]:
! pip install transformers torch pillow


  Using cached torch-2.5.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.1 MB 730.2 kB/s eta 0:00:14
   --- ------------------------------------ 0.8/10.1 MB 838.9 kB/s eta 0:00:12
   --- ------------------------------------ 0.8/10.1 MB 838.9 kB/s eta 0:00:12
   ---- ----------------------------------- 1.0/10.1 MB 774.8 kB/s eta 0:00:12
   ---- ----------------------------------- 1.0/10.1 MB 774.8 kB/s eta 0:00:12
   ---- ----------------------------------- 1.0/10.1 MB 774.8 kB/s eta 0:00:12
   ----- ------------------------------

In [3]:
from transformers import CLIPProcessor, CLIPModel
import torch
from PIL import Image

# Load the CLIP processor and model
image_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

c:\Users\hp\Downloads\anaconda\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--openai--clip-vit-base-patch32. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [4]:
# Load and preprocess the test image
example_image_path = "data/DAM/BOBYR1UXR42FR.jpeg"
example_image = Image.open(example_image_path).convert("RGB")
inputs = image_processor(images=example_image, return_tensors="pt", padding=True)
print("Input shape: ", inputs["pixel_values"].shape)


Input shape:  torch.Size([1, 3, 224, 224])


In [5]:
with torch.no_grad():
    embeddings = model.get_image_features(**inputs)

print("Embedding shape: ", embeddings.shape)


Embedding shape:  torch.Size([1, 512])


In [6]:
import os
dataset_dir = r"C:\Users\hp\Downloads\business_case\data\DAM"


dataset_image_paths = [
    os.path.join(dataset_dir, file_name) 
    for file_name in os.listdir(dataset_dir) 
    if file_name.endswith( ".jpeg")  
]

dataset_embeddings = []
for image_path in dataset_image_paths:  # List of paths to images in DAM
    image = Image.open(image_path).convert("RGB")
    inputs = image_processor(images=image, return_tensors="pt", padding=True)
    with torch.no_grad():
        embedding = model.get_image_features(**inputs)
        embedding = embedding / embedding.norm(p=2, dim=-1)  # Normalize
        dataset_embeddings.append(embedding)
dataset_embeddings = torch.cat(dataset_embeddings, dim=0) 

In [7]:

print("First Embedding:", dataset_embeddings[0])

print("Shape of Dataset Embeddings:", dataset_embeddings.shape)

First Embedding: tensor([-6.6985e-03,  1.5855e-02,  3.0665e-02,  9.3990e-03, -2.2298e-03,
        -7.9540e-03, -1.6985e-02,  3.2244e-02,  1.0605e-02,  3.3693e-02,
         1.7372e-02,  2.1614e-02,  5.6295e-02,  6.7058e-03,  3.8906e-03,
        -1.6884e-02,  1.7670e-02,  1.7991e-02,  2.0361e-02, -2.3091e-02,
        -4.4840e-02,  1.3795e-02, -1.3359e-03, -2.4528e-02,  2.1837e-03,
        -4.9338e-03, -2.1983e-02,  1.8358e-03, -1.0871e-02, -2.2823e-02,
        -1.0700e-02,  1.9207e-02, -2.5902e-03,  2.1941e-02, -2.4570e-02,
         9.7078e-03, -2.7557e-02, -2.0960e-03, -3.3443e-02,  1.5166e-01,
        -2.9828e-02, -2.3018e-02,  1.8851e-02,  1.4818e-02,  3.9122e-02,
        -1.2686e-01, -2.5003e-02,  1.2786e-02,  4.5006e-02, -1.4196e-02,
         2.0450e-02, -1.3926e-02, -3.5252e-03, -1.6192e-02, -5.4426e-02,
         3.0298e-02,  3.5920e-02, -1.1882e-02,  1.0940e-02,  3.5310e-02,
         2.9461e-02,  2.1959e-02,  2.3828e-02, -4.1658e-03, -4.9126e-02,
        -1.2922e-02,  3.1754e-02, 

In [9]:
import os
dataset_test_dir = r"C:\Users\hp\Downloads\business_case\data\test_image_headmind"


dataset_imagetests_paths = [
    os.path.join(dataset_test_dir, file_name) 
    for file_name in os.listdir(dataset_test_dir) 
    if file_name.endswith( ".jpg")  
]

dataset_test_embeddings = []
for image_path in dataset_imagetests_paths:  # List of paths to images in test_image_headminf
    image = Image.open(image_path).convert("RGB")
    inputs = image_processor(images=image, return_tensors="pt", padding=True)
    with torch.no_grad():
        embedding = model.get_image_features(**inputs)
        embedding = embedding / embedding.norm(p=2, dim=-1)  # Normalize
        dataset_test_embeddings.append(embedding)
dataset_test_embeddings = torch.cat(dataset_test_embeddings, dim=0) 

In [10]:
print("First Embedding:", dataset_test_embeddings[0])

print("Shape of Dataset Embeddings:", dataset_test_embeddings.shape)

First Embedding: tensor([ 2.1474e-02,  2.1318e-02, -6.7160e-03, -1.9440e-02,  1.2665e-02,
         1.6140e-02, -7.4059e-04,  6.4899e-02,  2.1913e-02,  2.7397e-02,
         5.2414e-02, -7.5704e-03, -8.9766e-03, -2.6601e-02,  4.1797e-02,
        -2.8958e-02,  1.3195e-02, -4.4427e-02, -1.5560e-02, -2.9687e-02,
        -1.9518e-02, -3.3603e-02,  5.1450e-02,  8.9399e-03, -1.5435e-02,
         2.4931e-02, -1.7235e-02, -1.3631e-02, -1.8597e-02,  1.0010e-02,
         4.2684e-03,  1.4087e-02, -3.2209e-02, -4.1422e-02, -5.2231e-02,
         1.5532e-02, -5.3910e-02,  4.2976e-03,  1.6531e-02,  1.3717e-01,
        -4.5154e-03, -2.5799e-03,  1.6476e-02, -2.7785e-02,  3.5370e-02,
        -8.8775e-02,  1.6845e-02,  1.1250e-02, -2.1148e-02,  3.7960e-02,
         2.4473e-02, -2.8179e-02,  4.1788e-02, -4.5632e-02,  1.3875e-03,
         2.3110e-02,  2.9239e-02, -9.9743e-03, -1.8455e-02,  8.9430e-03,
        -2.9610e-02, -3.2382e-02,  2.9840e-02, -2.0203e-02, -3.5311e-02,
        -6.3151e-03, -3.2055e-03, 

In [12]:
import os

# Path to your test images folder
dataset_test_dir = r"C:\Users\hp\Downloads\business_case\data\test_image_headmind"

# List all files in the directory
test_image_files = os.listdir(dataset_test_dir)
print("Test Image Files:")
print(test_image_files)


Test Image Files:
['image-20210928-102713-12d2869d.jpg', 'image-20210928-102718-2474636a.jpg', 'image-20210928-102721-8eaea48f.jpg', 'image-20210928-102725-7e28b44c.jpg', 'image-20210928-102729-f53d9faf.jpg', 'image-20210928-102732-8502aded.jpg', 'image-20210928-102747-05631ad4.jpg', 'image-20210928-102758-fd41d453.jpg', 'image-20210928-102802-39d3c54f.jpg', 'image-20210928-102806-79d7094b.jpg', 'image-20210928-102940-20cd277d.jpg', 'image-20210928-103146-c8d2fedb.jpg', 'image-20210928-103157-a46cc321.jpg', 'image-20210928-103200-ce51703f.jpg', 'image-20210928-103217-38e9a47d.jpg', 'image-20210928-103240-bd2d326b.jpg', 'image-20210928-103700-a406b29c.jpg', 'image-20210928-103707-a441ed0e.jpg', 'image-20210928-103731-9f1c8777.jpg', 'image-20210928-103742-55a72762.jpg', 'image-20210928-103754-89c16e1e.jpg', 'image-20210928-103807-7573be37.jpg', 'image-20210928-103810-3cb15816.jpg', 'image-20210928-103825-be7929ac.jpg', 'IMG_6875.jpg', 'IMG_6877.jpg', 'IMG_6878.jpg', 'IMG_6880.jpg', 'IMG_

In [11]:
import torch

# Function to compute cosine similarity scores
def compute_scores(emb_one, emb_two):
    
    scores = torch.nn.functional.cosine_similarity(emb_one, emb_two)
    return scores

# Function to fetch the most similar images for each test image
def fetch_similar(test_embeddings, dataset_embeddings, dataset_image_paths, top_k=1):
    """
    Fetches the `top_k` most similar images for each test image.
    - test_embeddings: Embeddings for test images.
    - dataset_embeddings: Embeddings for the main dataset.
    - dataset_image_paths: List of image paths in the main dataset.
    - top_k: Number of similar images to retrieve for each test image.
    """
    results = []

    for i, test_embedding in enumerate(test_embeddings):
        # Compute similarity scores with all dataset images
        scores = compute_scores(test_embedding.unsqueeze(0), dataset_embeddings)
        
        # Sort scores in descending order and get indices
        top_k_indices = torch.argsort(scores, descending=True)[:top_k]
        
        # Retrieve the top_k image paths and their scores
        top_k_images = [(dataset_image_paths[idx], scores[idx].item()) for idx in top_k_indices]
        
        results.append({
            "test_image_index": i,
            "similar_images": top_k_images
        })

    return results

# Example usage
top_k = 5  # Number of similar images to retrieve
results = fetch_similar(
    test_embeddings=dataset_test_embeddings,
    dataset_embeddings=dataset_embeddings,
    dataset_image_paths=dataset_image_paths,
    top_k=top_k
)

# Print results for each test image
for result in results:
    print(f"Test Image Index: {result['test_image_index']}")
    for similar_image in result['similar_images']:
        print(f"  Similar Image: {similar_image[0]}, Score: {similar_image[1]:.4f}")


Test Image Index: 0
  Similar Image: C:\Users\hp\Downloads\business_case\data\DAM\M505SOAHGM900.jpeg, Score: 0.7885
  Similar Image: C:\Users\hp\Downloads\business_case\data\DAM\M0531NWDDM900.jpeg, Score: 0.7845
  Similar Image: C:\Users\hp\Downloads\business_case\data\DAM\M565SOAAUXM830.jpeg, Score: 0.7819
  Similar Image: C:\Users\hp\Downloads\business_case\data\DAM\M565SOLDPXM090.jpeg, Score: 0.7804
  Similar Image: C:\Users\hp\Downloads\business_case\data\DAM\M0447CANWM43R.jpeg, Score: 0.7727
Test Image Index: 1
  Similar Image: C:\Users\hp\Downloads\business_case\data\DAM\M0531NWDDM900.jpeg, Score: 0.8195
  Similar Image: C:\Users\hp\Downloads\business_case\data\DAM\M0505SNEAM900.jpeg, Score: 0.8112
  Similar Image: C:\Users\hp\Downloads\business_case\data\DAM\M565SOAAUXM830.jpeg, Score: 0.8099
  Similar Image: C:\Users\hp\Downloads\business_case\data\DAM\S0204OAHGXM830.jpeg, Score: 0.8099
  Similar Image: C:\Users\hp\Downloads\business_case\data\DAM\M531SOCYMXM900.jpeg, Score: 0.

In [17]:
test_image_index = 24
fifth_test_embedding = dataset_test_embeddings[test_image_index].unsqueeze(0)

# Compute similarity scores with all dataset embeddings
scores = torch.nn.functional.cosine_similarity(fifth_test_embedding, dataset_embeddings)

# Sort scores and get the top matches
sorted_scores, sorted_indices = torch.sort(scores, descending=True)

# Display the scores
print("Similarity Scores for Fifth Test Image:")
for i in range(5):  # Display top 5 most similar images
    similar_image_index = sorted_indices[i].item()
    similar_image_score = sorted_scores[i].item()
    similar_image_path = dataset_image_paths[similar_image_index]
    print(f"  Similar Image: {similar_image_path}, Score: {similar_image_score:.4f}")

Similarity Scores for Fifth Test Image:
  Similar Image: C:\Users\hp\Downloads\business_case\data\DAM\DSGTS1UXR10A0.jpeg, Score: 0.8633
  Similar Image: C:\Users\hp\Downloads\business_case\data\DAM\DSGTA1UXR10A0.jpeg, Score: 0.8553
  Similar Image: C:\Users\hp\Downloads\business_case\data\DAM\DSGTS2UXR20B0.jpeg, Score: 0.8411
  Similar Image: C:\Users\hp\Downloads\business_case\data\DAM\30MTBUXR55F1.jpeg, Score: 0.8391
  Similar Image: C:\Users\hp\Downloads\business_case\data\DAM\CLUBM4UWR45A0.jpeg, Score: 0.8358
